In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

In [2]:
# Try first with English corpus
with open('./current_corpora/en_wiki_extractor.txt', 'r', encoding='utf-8') as f:
    text = f.read()
    f.close()
    
# Encode text
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}
encoded = np.array([char2int[ch] for ch in text])


# One hot encode
def one_hot_encode(arr, n_labels):
    one_hot = np.zeros((np.multiply(*arr.shape), n_labels), dtype=np.float32)
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    return one_hot


# Batches
def get_batches(arr, n_seqs, n_steps):
    '''Create a generator that returns batches of size n_seqs x n_steps from arr.
    
        Args
        ----
        arr: Array to make batches from
        n_seqs: Batch size, number of sequences per batch
        n_steps: Number of sequence steps per batch
    '''
    
    batch_size = n_seqs * n_steps
    n_batches = len(arr)//batch_size
    
    # Make only full batches
    arr = arr[:n_batches * batch_size]
    
    # Reshape into n_seqs rows
    arr = arr.reshape((n_seqs, -1))
    
    for n in range(0, arr.shape[1], n_steps):
        
        # Features
        x = arr[:, n:n+n_steps]
        
        # Targets, shifted by 1
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+n_steps]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
            
        yield x, y

In [3]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_steps=100, n_hidden=256, n_layers=2, 
                 drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # Creating char dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        # Define LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        # Define dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        # Define fully connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
        
        # Initialize weights
        self.init_weights()
        
        
    def forward(self, x, hc):
        ''' Forward pass. Inputs are `x` and hidden/cell state are `hc`. '''
        
        # Get x and new hidden state (h, c) from LSTM
        x, (h, c) = self.lstm(x, hc)
        
        # Pass x through dropout layer
        x = self.dropout(x)
        
        # Stack up LSTM outputs
        x = x.reshape(x.size()[0]*x.size()[1], self.n_hidden)
        
        # Put x through fc layer
        x = self.fc(x)
        
        # Return x and hidden state (h, c)
        return x, (h, c)
    
    
    def predict(self, char, h=None, cuda=False, top_k=None):
        ''' Given a character, predict the next character.
            
            Returns predicted character and hidden state.
        '''
        
        if cuda:
            self.cuda()
        else:
            self.cpu()
            
        if h is None:
            h = self.init_hidden(1)
            
        x = np.array([[self.char2int[char]]])
        x = one_hot_encode(x, len(self.chars))
        inputs = torch.from_numpy(x)
        
        if cuda:
            inputs = inputs.cuda()
            
        h = tuple([each.data for each in h])
        out, h = self.forward(inputs, h)
        p = F.softmax(out, dim=1).data
        
        if cuda:
            p = p.cpu()
        
        if top_k is None:
            top_ch = np.arange(len(self.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
            
        return self.int2char[char], h
    
    def init_weights(self):
        ''' Initialize weights for fc layer '''
        
        initrange = 0.1
        
        # Set bias tensor to all zeros
        self.fc.bias.data.fill_(0)
        # FC weights as random uniform
        self.fc.weight.data.uniform_(-1, 1)
        
    def init_hidden(self, n_seqs):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x n_seqs x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        return (weight.new(self.n_layers, n_seqs, self.n_hidden).zero_(),
                weight.new(self.n_layers, n_seqs, self.n_hidden).zero_())

In [4]:
# # Train the model (without Cross Validation)

# def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, val_frac=0.1, cuda=False, print_every=10):
#     ''' Training a network
    
#         Args
#         ----
#         net: CharRNN network
#         data: text data to train on
#         epochs: Number of epochs to train
#         n_seqs: Number of sequences per batch (batch size)
#         n_steps: Number of character steps per batch
#         lr: learning rate
#         clip: gradient clipping
#         val_frac: Validation data as fraction
#         cuda: Using CUDA on a GPU
#         print_every: Number of steps for printing training/validation loss
#     '''
    
#     net.train()
    
#     opt = torch.optim.Adam(net.parameters(), lr=lr)
#     criterion = nn.CrossEntropyLoss()
    
#     # Split training/validation data
#     val_idx = int(len(data)*(1-val_frac))
#     data, val_data = data[:val_idx], data[val_idx:]
    
#     if cuda:
#         net.cuda()
        
#     counter = 0
#     n_chars = len(net.chars)
    
#     for e in range(epochs):
#         h = net.init_hidden(n_seqs)
        
#         for x, y in get_batches(data, n_seqs, n_steps):
#             counter += 1
            
#             # One-hot encode our data and make them Torch tensors
#             x = one_hot_encode(x, n_chars)
#             inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
#             if cuda:
#                 inputs, targets = inputs.cuda(), targets.cuda()

#             # Creating new variables for the hidden state, otherwise
#             # we'd backprop through the entire training history
#             h = tuple([each.data for each in h])

#             net.zero_grad()
#             output, h = net.forward(inputs, h)
#             loss = criterion(output, targets.view(n_seqs*n_steps).type(torch.LongTensor))
#             loss.backward()
            
#             # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
#             nn.utils.clip_grad_norm_(net.parameters(), clip)

#             opt.step()
            
#             if counter % print_every == 0:
                
#                 # Get validation loss
#                 val_h = net.init_hidden(n_seqs)
#                 val_losses = []
                
#                 for x, y in get_batches(val_data, n_seqs, n_steps):
                    
#                     # One-hot encode our data and make them Torch tensors
#                     x = one_hot_encode(x, n_chars)
#                     x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
#                     # Creating new variables for the hidden state, otherwise
#                     # we'd backprop through the entire training history
#                     val_h = tuple([each.data for each in val_h])
                    
#                     inputs, targets = x, y
#                     if cuda:
#                         inputs, targets = inputs.cuda(), targets.cuda()

#                     output, val_h = net.forward(inputs, val_h)
#                     val_loss = criterion(output, targets.view(n_seqs*n_steps).type(torch.LongTensor))
                
#                     val_losses.append(val_loss.item())
                
#                 print("Epoch: {}/{}...".format(e+1, epochs),
#                       "Step: {}...".format(counter),
#                       "Loss: {:.4f}...".format(loss.item()),
#                       "Val Loss: {:.4f}".format(np.mean(val_losses)))
                

# net = CharRNN(chars, n_hidden=512, n_layers=2)
# print(net)

In [5]:
# # Train without Cross Validation

# # Hyperparams
# n_seqs, n_steps = 128, 100

# train(net, encoded, epochs=25, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, cuda=False, print_every=10)

In [6]:
# Train the model (with Cross Validation)     
   
def train(net, data, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, cuda=False):
    ''' Training a network
    
        Args
        ----
        net: CharRNN network
        data: text data to train on
        epochs: Number of epochs to train
        n_seqs: Number of sequences per batch (batch size)
        n_steps: Number of character steps per batch
        lr: learning rate
        clip: gradient clipping
        cuda: Using CUDA on a GPU
    '''
    
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    if cuda:
        net.cuda()
        
    n_chars = len(net.chars)
    
    for e in range(epochs):
        h = net.init_hidden(n_seqs)
        
        for x, y in get_batches(data, n_seqs, n_steps):
            
            # One hot encode and convert to Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if cuda:
                inputs, targets = inputs.cuda(), targets.cuda()

            # Create new variables for the hidden state to avoid backprop
            # through the entire training history
            h = tuple([each.data for each in h])

            net.zero_grad()
            output, h = net.forward(inputs, h)

            if cuda:
              loss = criterion(output, targets.view(n_seqs*n_steps).type(torch.cuda.LongTensor))
            else:
              loss = criterion(output, targets.view(n_seqs*n_steps).type(torch.LongTensor))

            loss.backward()
            
            # `clip_grad_norm` to avoid exploding gradient
            nn.utils.clip_grad_norm_(net.parameters(), clip)

            opt.step()
            
        print("Epoch: {}/{}...".format(e+1, epochs),
            "Loss: {:.4f}...".format(loss.item()))


# KFold Cross-Validation
def cross_validate(net, data, n_folds=5, epochs=10, n_seqs=10, n_steps=50, lr=0.001, clip=5, cuda=False):
    ''' Perform cross-validation on the network.
    
        Args
        ----
        net: CharRNN network
        data: text data to train on
        n_folds: Number of folds for cross-validation
        epochs: Number of epochs to train
        n_seqs: Number of sequences per batch (batch size)
        n_steps: Number of character steps per batch
        lr: learning rate
        clip: gradient clipping
        cuda: Using CUDA on a GPU
    '''
    
    kf = KFold(n_splits=n_folds, shuffle=True)
    criterion = nn.CrossEntropyLoss()
    
    for fold, (train_idx, val_idx) in enumerate(kf.split(data)):
        
        train_data, val_data = data[train_idx], data[val_idx]
        
        fold_net = CharRNN(net.chars, n_hidden=net.n_hidden, n_layers=net.n_layers, drop_prob=net.drop_prob, lr=net.lr)
        if cuda:
            fold_net.cuda()
        
        train(fold_net, train_data, epochs, n_seqs, n_steps, lr, clip, cuda=cuda)
        
        # Evaluate on validation data
        val_loss = 0.0
        val_h = fold_net.init_hidden(n_seqs)
        fold_net.eval()
        with torch.no_grad():
            for x, y in get_batches(val_data, n_seqs, n_steps):
                x = one_hot_encode(x, len(net.chars))
                inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
                if cuda:
                    inputs, targets = inputs.cuda(), targets.cuda()
                output, val_h = fold_net.forward(inputs, val_h)

                if cuda:
                  val_loss += criterion(output, targets.view(n_seqs * n_steps).type(torch.cuda.LongTensor))
                else:
                  val_loss += criterion(output, targets.view(n_seqs * n_steps).type(torch.LongTensor))
        
        val_loss /= len(val_data)
        val_perplexity = torch.exp(val_loss)
        print("Fold: {}/{}...".format(fold + 1, n_folds),
            "Validation Loss: {:.4f}...".format(val_loss.item()),
            "Validation Perplexity: {:.4f}".format(val_perplexity.item()))

        
net = CharRNN(chars, n_hidden=512, n_layers=2)
print(net)

CharRNN(
  (lstm): LSTM(28, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5, inplace=False)
  (fc): Linear(in_features=512, out_features=28, bias=True)
)


In [ ]:
# Train with Cross Validation

# Hyperparams
n_seqs, n_steps = 10, 1

cross_validate(net, encoded, n_folds=5, epochs=10, n_seqs=n_seqs, n_steps=n_steps, lr=0.001, clip=5, cuda=True)

In [7]:
# # Save the model

# model_name = 'char_rnn.net'

# checkpoint = {'n_hidden': net.n_hidden,
#               'n_layers': net.n_layers,
#               'state_dict': net.state_dict(),
#               'tokens': net.chars}

# with open(model_name, 'wb') as f:
#     torch.save(checkpoint, f)

In [8]:
# Test/Sample model

# def sample(net, size, prime='The', top_k=None, cuda=False):
     
#     if cuda:
#         net.cuda()
#     else:
#         net.cpu()
#     net.eval()
    
#     # Run through the prime chars
#     chars = [ch for ch in prime]
    
#     h = net.init_hidden(1)
#     for ch in prime:
#         char, h = net.predict(ch, h, cuda=cuda, top_k=top_k)

#     chars.append(char)
    
#     # Pass prev char and get new one
#     for ii in range(size):
#         char, h = net.predict(chars[-1], h, cuda=cuda, top_k=top_k)
#         chars.append(char)

#     return ''.join(chars)

In [9]:
# Load a trained model

# with open('modelname.net', 'rb') as f:
#     checkpoint = torch.load(f)
    
# loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
# loaded.load_state_dict(checkpoint['state_dict'])

# print(sample(loaded, 2000, cuda=False, top_k=5, prime="Example text"))